# Save datasets to a ROOT file after processing

With RDataFrame, you can read your dataset, add new columns with processed values and finally use `Snapshot` to save the resulting data to a ROOT file in TTree format.

In [1]:
import ROOT

df = ROOT.RDataFrame("dataset","../../data/example_file.root")
df1 = df.Define("c","a+b")

out_treename = "outtree"
out_filename = "outtree.root"
out_columns = ["a","b","c"]
snapdf = df1.Snapshot(out_treename, out_filename, out_columns)

/opt/homebrew/Cellar/root/6.38.00/lib/root/cppyy/__init__.py:374: UserWarning: CPyCppyy API not found (tried: /Users/chanayo/.pyenv/versions/3.14.0/include/site/python3.14); set CPPYY_API_PATH envar to the 'CPyCppyy' API directory to fix
  warnings.warn("CPyCppyy API not found (tried: %s); "
Info in <[ROOT.RDF] Info /private/tmp/root-20260120-18625-wjnpn/root-6.38.00/builddir/include/ROOT/RDF/RInterface.hxx:1363 in auto ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager, void>::Snapshot(std::string_view, std::string_view, const ColumnNames_t &, const RSnapshotOptions &)::(anonymous class)::operator()() const [Proxied = ROOT::Detail::RDF::RLoopManager, DataSource = void]>: 
	In ROOT 6.38, the default compression settings of Snapshot have been changed from 101 (ZLIB with compression level 1, the TTree default) to 505 (ZSTD with compression level 5). This change may result in smaller Snapshot output dataset size by default. In order to suppress this message, set 'ROOT_RDF_SNAPSHOT_INFO

We can now check that the dataset was correctly stored in a file:

In [2]:
%%bash
rootls -lt outtree.root

TTree  Jan 28 11:25 2026 outtree;1 "outtree" 
  a  "a/D"  16071
  b  "b/D"  16071
  c  "c/D"  16071
  Cluster INCLUSIVE ranges:
   - # 0: [0, 1999]
  The total number of clusters is 1


Result of a Snapshot is still an RDataFrame that can be further used:

In [3]:
snapdf.Display().Print()

+-----+-----------+----------+-----------+
| Row | a         | b        | c         | 
+-----+-----------+----------+-----------+
| 0   | 0.977711  | 0.999742 | 1.977453  | 
+-----+-----------+----------+-----------+
| 1   | 2.280201  | 0.484974 | 2.765175  | 
+-----+-----------+----------+-----------+
| 2   | 0.563482  | 0.392314 | 0.955796  | 
+-----+-----------+----------+-----------+
| 3   | 3.042156  | 0.333539 | 3.375695  | 
+-----+-----------+----------+-----------+
| 4   | 28.574399 | 0.648126 | 29.222524 | 
+-----+-----------+----------+-----------+


# Cutflow reports
Filters applied to the dataset can be given a name. The `Report` method will gather information about filter efficiency and show the data flow between subsequent cuts on the original dataset.


In [4]:
df = ROOT.RDataFrame("sig_tree", "https://root.cern/files/Higgs_data.root")

filter1 = df.Filter("lepton_eta > 0", "Lepton eta cut")
filter2 = filter1.Filter("lepton_phi < 1", "Lepton phi cut")

rep = df.Report()
rep.Print()

Lepton eta cut: pass=4880       all=10000      -- eff=48.80 % cumulative eff=48.80 %
Lepton phi cut: pass=3842       all=4880       -- eff=78.73 % cumulative eff=38.42 %


# Using C++ functions in Python
- We still want to perform complex operations in Python but plain Python code is prone to be slow and not thread-safe. 

- Instead, you can inject C++ functions that will do the work in your event loop during runtime. 

- This mechanism uses the C++ interpreter `cling` shipped with ROOT, making this possible in a single line of code. 

- Let's start by defining a function that will allow us to change the type of a the RDataFrame dataset entry numbers (stored in the special column "rdfentry") from `unsigned long long` to `float`.

In [6]:
%%cpp

float asfloat(unsigned long long entrynumber){
    return entrynumber;
}

Then let's define another function that takes a `float` values and computes its square.

In [7]:
%%cpp

float square(float val){
    return val * val;
}

And now let's use these functions with RDataFrame! 

We start by creating an empty RDataFrame with 100 consecutive entries and defining new columns on it:

In [8]:
# Create a new RDataFrame from scratch with 100 consecutive entries
df = ROOT.RDataFrame(100)

# Create a new column using the previously declared C++ functions
df1 = df.Define("a", "asfloat(rdfentry_)")
df2 = df1.Define("b", "square(a)")

We can now plot the values of the columns in a graph:

In [9]:
# Show the two columns created in a graph
c = ROOT.TCanvas()
graph = df2.Graph("a","b")
graph.SetMarkerStyle(20)
graph.SetMarkerSize(0.5)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle("My graph")
graph.Draw("AP")
c.Draw()

# Using all cores of your machine with multi-threaded RDataFrame
- RDataFrame can transparently perform multi-threaded event loops to speed up the execution of its actions. 

- Users have to call `ROOT::EnableImplicitMT()` before constructing the RDataFrame object to indicate that it should take advantage of a pool of worker threads. 

- Each worker thread processes a distinct subset of entries, and their partial results are merged before returning the final values to the user.

- RDataFrame operations such as Histo1D or Snapshot are guaranteed to work correctly in multi-thread event loops. 

- User-defined expressions, such as strings or lambdas passed to `Filter`, `Define`, `Foreach`, `Reduce` or `Aggregate` will have to be thread-safe, i.e. it should be possible to call them concurrently from different threads.

In [10]:
# Activate multithreading capabilities
# By default takes all available cores on the machine
ROOT.EnableImplicitMT()

treename = "Events"
filename = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
df = ROOT.RDataFrame(treename, filename)

df.Sum("nMuon").GetValue()

# Disable implicit multithreading when done
ROOT.DisableImplicitMT()